In [1]:
## imports
import pandas as pd
import numpy as np
import plotnine
from plotnine import *
import random

## print multiple things from same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from datetime import datetime, timedelta

## Load data

In [2]:
## load data on 2020 crimes in DC
dc_crim_2020 = pd.read_csv("https://opendata.arcgis.com/datasets/f516e0dd7b614b088ad781b0c4002331_2.csv")
dc_crim_2020.head()
dc_crim_2020.shape
dc_crim_2020.info()


,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID
0,-76.970922,38.921043,20165463,2020/11/19 19:02:05+00,DAY,OTHERS,THEFT F/AUTO,2200 - 2299 BLOCK OF 25TH PLACE NE,402521.83,139240.76,...,011100 1,11100.0,Precinct 71,38.921035,-76.970919,NaN,2020/11/19 18:29:50+00,2020/11/19 18:35:19+00,98176975,NaN
1,-76.995572,38.888685,20165470,2020/11/19 19:32:37+00,DAY,OTHERS,THEFT F/AUTO,700 - 799 BLOCK OF A STREET SE,400384.35,135648.37,...,006600 1,6600.0,Precinct 89,38.888677,-76.995570,NaN,2020/11/19 18:56:08+00,NaN,98176976,NaN
2,-77.012847,38.919183,20165473,2020/11/19 19:33:19+00,DAY,OTHERS,MOTOR VEHICLE THEFT,100 - 138 BLOCK OF W STREET NW,398886.09,139033.94,...,003301 3,3301.0,Precinct 135,38.919175,-77.012845,NaN,2020/11/19 18:40:26+00,2020/11/19 19:33:39+00,98176977,NaN
3,-77.012839,38.951767,20165480,2020/11/19 20:25:33+00,EVENING,OTHERS,THEFT F/AUTO,5000 - 5099 BLOCK OF NEW HAMPSHIRE AVENUE NW,398887.26,142651.05,...,002202 1,2202.0,Precinct 57,38.951759,-77.012837,NaN,2020/11/19 19:00:00+00,2020/11/19 19:35:44+00,98176978,NaN
4,-76.984574,38.892849,20165483,2020/11/19 20:24:18+00,EVENING,OTHERS,THEFT/OTHER,200 - 299 BLOCK OF 14TH PLACE NE,401338.40,136110.77,...,008002 2,8002.0,Precinct 86,38.892842,-76.984572,NaN,2020/11/19 19:20:38+00,2020/11/19 19:24:55+00,98176979,NaN


(27915, 25)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27915 entries, 0 to 27914
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   X                     27915 non-null  float64
 1   Y                     27915 non-null  float64
 2   CCN                   27915 non-null  int64  
 3   REPORT_DAT            27915 non-null  object 
 4   SHIFT                 27915 non-null  object 
 5   METHOD                27915 non-null  object 
 6   OFFENSE               27915 non-null  object 
 7   BLOCK                 27915 non-null  object 
 8   XBLOCK                27915 non-null  float64
 9   YBLOCK                27915 non-null  float64
 10  WARD                  27915 non-null  int64  
 11  ANC                   27915 non-null  object 
 12  DISTRICT              27886 non-null  float64
 13  PSA                   27880 non-null  float64
 14  NEIGHBORHOOD_CLUSTER  27915 non-null  object 
 15  BLOCK_GROUP        

### Loop approach

In [113]:
## define crimes to look for and crimes to look within
CCN_examples = ['20165648', '20123250']
crimes_lookfor = dc_crim_2020.loc[dc_crim_2020.CCN.astype(str).isin(CCN_examples),
                ['CCN', 'WARD', 'OFFENSE', 'report_dt']].copy()
other_crimes = dc_crim_2020[~dc_crim_2020.CCN.astype(str).isin(CCN_examples)].copy()


## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i in range(0, crimes_lookfor.shape[0]):
    
    ## extract row
    one_row = crimes_lookfor.iloc[i]
    
    ## first, subset to crimes in same ward
    same_wards = other_crimes[other_crimes.WARD == one_row.WARD].copy()
    print(same_wards.WARD.value_counts())
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    cutoff = one_row.report_dt +  timedelta(minutes=20)
    print(one_row.report_dt)
    print(cutoff)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= one_row.report_dt) & 
                                    (same_wards.report_dt <= cutoff)].copy()
    print(same_wards_sametime)
    
    ## third, store the results
    store_matches[str(one_row.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
all_matches.head()

## bad part

6    4752
Name: WARD, dtype: int64
2020-11-19 21:25:50+00:00
2020-11-19 21:45:50+00:00
               X          Y       CCN              REPORT_DAT    SHIFT  \
13463 -77.023972  38.907912  20165497  2020/11/19 21:38:59+00  EVENING   

       METHOD              OFFENSE                               BLOCK  \
13463  OTHERS  MOTOR VEHICLE THEFT  1300 - 1399 BLOCK OF 9TH STREET NW   

         XBLOCK    YBLOCK  ...  BID              START_DATE  \
13463  397921.0  137783.0  ...  NaN  2020/11/19 14:45:25+00   

                     END_DATE   OBJECTID OCTO_RECORD_ID is_after_christmas  \
13463  2020/11/19 16:03:07+00  587047293    20165497-01              False   

       is_theft is_theft_notmotor  summary_shifts                 report_dt  
13463      True             False       nighttime 2020-11-19 21:38:59+00:00  

[1 rows x 30 columns]
2    4360
Name: WARD, dtype: int64
2020-08-29 01:00:25+00:00
2020-08-29 01:20:25+00:00
Empty DataFrame
Columns: [X, Y, CCN, REPORT_DAT, SHIFT, METHOD, O

,,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID,is_after_christmas,is_theft,is_theft_notmotor,summary_shifts,report_dt
20165648,13463,-77.023972,38.907912,20165497,2020/11/19 21:38:59+00,EVENING,OTHERS,MOTOR VEHICLE THEFT,1300 - 1399 BLOCK OF 9TH STREET NW,397921.0,137783.0,...,NaN,2020/11/19 14:45:25+00,2020/11/19 16:03:07+00,587047293,20165497-01,False,True,False,nighttime,2020-11-19 21:38:59+00:00


### Function approach

In [136]:
def proximate_crimes(search_for: pd.DataFrame,
                    search_in: pd.DataFrame):
    
    ## first, subset to crimes in same ward
    same_wards = search_in[search_in.WARD == search_for.WARD].copy()
    print(same_wards.WARD.value_counts())
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ### substep: get time cutoff
    cutoff = search_for.report_dt +  timedelta(minutes=20)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= search_for.report_dt) & 
                                    (same_wards.report_dt <= cutoff)].copy()
    
    ## add focal match
    same_wards_sametime['focal_crime'] = search_for.CCN
    
    ## return
    return(same_wards_sametime)
    

In [138]:
one_match = proximate_crimes(search_for = crimes_lookfor.iloc[0],
                            search_in = other_crimes)
one_match

all_matches_list = [proximate_crimes(crimes_lookfor.iloc[i],
                               other_crimes) for i in range(0, crimes_lookfor.shape[0])]
all_matches = pd.concat(all_matches_list)
all_matches # happens to only be one match

6    4752
Name: WARD, dtype: int64


,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID,is_after_christmas,is_theft,is_theft_notmotor,summary_shifts,report_dt,focal_crime
13463,-77.023972,38.907912,20165497,2020/11/19 21:38:59+00,EVENING,OTHERS,MOTOR VEHICLE THEFT,1300 - 1399 BLOCK OF 9TH STREET NW,397921.0,137783.0,...,2020/11/19 14:45:25+00,2020/11/19 16:03:07+00,587047293,20165497-01,False,True,False,nighttime,2020-11-19 21:38:59+00:00,20165648


6    4752
Name: WARD, dtype: int64
2    4360
Name: WARD, dtype: int64


,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID,is_after_christmas,is_theft,is_theft_notmotor,summary_shifts,report_dt,focal_crime
13463,-77.023972,38.907912,20165497,2020/11/19 21:38:59+00,EVENING,OTHERS,MOTOR VEHICLE THEFT,1300 - 1399 BLOCK OF 9TH STREET NW,397921.0,137783.0,...,2020/11/19 14:45:25+00,2020/11/19 16:03:07+00,587047293,20165497-01,False,True,False,nighttime,2020-11-19 21:38:59+00:00,20165648
